**Note:**

If having trouble seeing the simulated outputs (e.g., ``Button(button_style='success', description=' Simulate...'``, but not the acutally button), you propbably don't have the needed widgets (i.e., if you look at your Jupyter Notebook's menu bar, you probably don't have a `Widgets` between `Kernel` and `Help`), check out this website:
https://github.com/jupyter-widgets/ipywidgets/issues/1702

If you see something like:
```
A Jupyter widget could not be displayed because the widget state could not be found. This could happen if the kernel storing the widget is no longer available, or if the widget state was not saved in the notebook. You may be able to create the widget by running the appropriate cells.
```
Then restart the kernel and run all.

In [1]:
# Interactive module for a simple system with just one aerated CSTR,
# system developed by Joy and the interactive module by Yalin

import numpy as np
from matplotlib import pyplot as plt
from biosteam import System
from qsdsan import sanunits as su
from qsdsan import processes as pc
from qsdsan import WasteStream, set_thermo

# =============================================================================
# Benchmark Simulation Model No. 1
# =============================================================================

############# load components and set thermo #############
cmps = pc.load_asm1_cmps()
set_thermo(cmps)

############# create WasteStream objects #################
Q = 18446           # influent flowrate [m3/d]
Temp = 273.15+15    # temperature [K]

PE = WasteStream('primary_effluent', T=Temp)
PE.set_flow_by_concentration(Q,
                              {'S_S':69.5,
                              'X_BH':28.17,
                              'X_S':202.32,
                              'X_I':51.2,
                              'S_NH':31.56,
                              'S_I':30,
                              'S_ND':6.95,
                              'X_ND':10.59,
                              'S_ALK':7*12},
                              units=('m3/d', 'mg/L'))

SE = WasteStream('secondary_effluent', T=Temp)
# WAS = WasteStream('waste_activated_sludge', T=Temp)
RE = WasteStream('recycled_effluent', T=Temp)
# RAS = WasteStream('recycled_activated_sludge', T=Temp)

############# load and tailor process models #############
# V_an = 1000    # anoxic zone tank volume
V_ae = 1333    # aerated zone tank volume
# Q_was = 385    # sludge wastage flowrate
# Q_ras = 18446    # recycle sludge flowrate

aer = pc.DiffusedAeration('Fixed_Aeration', 'S_O', KLa_20=240, SOTE=0.3,
                          T_air=Temp, T_water=Temp, d_submergence=4-0.3)
asm1 = pc.ASM1(components=cmps,
               Y_A=0.24, Y_H=0.67, f_P=0.08, i_XB=0.08, i_XP=0.06,
               mu_H=4.0, K_S=10.0, K_O_H=0.2, K_NO=0.5, b_H=0.3,
               eta_g=0.8, eta_h=0.8, k_h=3.0, K_X=0.1, mu_A=0.5,
               K_NH=1.0, b_A=0.05, K_O_A=0.4, k_a=0.05)
# 
#%%
############# create unit operations #####################
# A1 = su.CSTR('Anoxic_1', ins=[PE, RE, RAS], V_max=V_an,
#               aeration=None, suspended_growth_model=asm1)

# A2 = su.CSTR('Anoxic_2', A1-0, V_max=V_an,
#               aeration=None, suspended_growth_model=asm1)

O1 = su.CSTR('Aerobic_1', [PE, RE], V_max=V_ae, aeration=aer,
              DO_ID='S_O', suspended_growth_model=asm1)

# O2 = su.CSTR('Aerobic_2', O1-0, V_max=V_ae, aeration=aer,
#               DO_ID='S_O', suspended_growth_model=asm1)

# O3 = su.CSTR('Aerobic_3', O2-0, V_max=V_ae, aeration=2.0,
#               DO_ID='S_O', suspended_growth_model=asm1)


S1 = su.Splitter('S1', O1-0, [RE, SE], split=0.6, init_with='WasteStream')

# C1 = su.FlatBottomCircularClarifier('C1', S1-1, [SE, 'sludge'],
#                                     sludge_flow_rate=Q_ras+Q_was, surface_area=1500,
#                                     height=4, N_layer=10, feed_layer=4,
#                                     # height=12, N_layer=3, feed_layer=2,
#                                     X_threshold=3000, v_max=474, v_max_practical=250,
#                                     rh=5.76e-4, rp=2.86e-3, fns=2.28e-3)


# S2 = su.Splitter('S2', C1-1, [RAS, WAS], split=Q_ras/(Q_ras+Q_was), init_with='WasteStream')

############# system simulation ############################

sys = System('sys', path=(O1, S1), recycle=(RE,))
# bsm1 = System('BSM1', path=(bio, S1, C2, S2), recycle=(RE, RAS))

#%%
def run(t, method, set_init, **kwargs):
    if set_init:
        # A1.set_init_conc(S_I=30.0, S_S=5.0, X_I=1000, X_S=100, X_BH=500, X_BA=100, X_P=100,
        #                   S_O=2.0, S_NH=2.0, S_ND=1.0, X_ND=1.0, S_NO=20.0, S_ALK=7*12)
        # A2.set_init_conc(S_I=30.0, S_S=5.0, X_I=1000, X_S=100, X_BH=500, X_BA=100, X_P=100,
        #                   S_O=2.0, S_NH=2.0, S_ND=1.0, X_ND=1.0, S_NO=20.0, S_ALK=7*12)
        O1.set_init_conc(S_I=30.0, S_S=5.0, X_I=1000, X_S=100, X_BH=500, X_BA=100, X_P=100,
                          S_O=2.0, S_NH=2.0, S_ND=1.0, X_ND=1.0, S_NO=20.0, S_ALK=7*12)
        # O2.set_init_conc(S_I=30.0, S_S=5.0, X_I=1000, X_S=100, X_BH=500, X_BA=100, X_P=100,
        #                   S_O=2.0, S_NH=2.0, S_ND=1.0, X_ND=1.0, S_NO=20.0, S_ALK=7*12)
        # O3.set_init_conc(S_I=30.0, S_S=5.0, X_I=1000, X_S=100, X_BH=500, X_BA=100, X_P=100,
        #                   S_O=2.0, S_NH=2.0, S_ND=1.0, X_ND=1.0, S_NO=20.0, S_ALK=7*12)
        # C1.set_init_solubles(S_I=30, S_S=5.0, S_O=2.0, S_NO=20, S_NH=2.0, S_ALK=7*12)
        # C1.set_init_TSS([10, 20, 40, 70, 200, 300, 350, 350, 2000, 4000])
        # S1.set_init_conc(S_I=30.0, S_S=5.0, X_I=1000, X_S=100, X_BH=500, X_BA=100,
        #                   S_O=2.0, S_NH=2.0, S_ND=1.0, X_ND=1.0, S_NO=20.0, S_ALK=7*12)
        # S2.set_init_conc(S_I=30.0, S_S=5.0, X_I=1000, X_S=100, X_BH=500, X_BA=100,
        #                   S_O=2.0, S_NH=2.0, S_ND=1.0, X_ND=1.0, S_NO=20.0, S_ALK=7*12)
    else:
        # for u in (A1, A2, O1, O2, O3):
        #     u._concs = None
        # C1._solubles = C1._solids = None
        pass
    sys.reset_cache()
    sys.simulate(t_span = (0, t), method=method, **kwargs)
    

def plot(variables, txt_file_path=None):
    fig, ax = plt.subplots(figsize=(8, 4.5)) # 16:9
    txt_file_path = 'sol.txt' if not txt_file_path else txt_file_path
    data = np.loadtxt(txt_file_path)
    x = data[:, 0] # time
    for i in variables: # state variables of interest
        ID = i.split(' ')[0]
        y = data[:, cmps.index(ID)+1] # the first one is time
        ax.plot(x, y, '-o', label=ID)
    
    ax.legend(loc='best')
    ax.set(xlabel='Time [d]', ylabel='Concentration [mg COD/L] or [mg/L]')

In [2]:
from ipywidgets.widgets import Label, Text, Button, Dropdown, SelectMultiple
t_lbl = Label('Simulation time [d]')
display(t_lbl)
t_txt = Text('1')
display(t_txt)

t_step_lbl = Label('Simulation time step [d]')
display(t_step_lbl)
t_step_txt = Text('0.05')
display(t_step_txt)

method_menu = Dropdown(
    options=['RK45', 'RK23', 'DOP853', 'BDF', 'LSODA'],
    value='BDF',
    description='ODE solver:',
    disabled=False,
)
display(method_menu)

var_lbl = Label('Choose one or more (use shift and/or ctrl/cmd) state variables of interest:')
display(var_lbl)
opts = [f'{i.ID} ({i.description.lower()})' for i in cmps]
var_slt = SelectMultiple(
    options=opts,
    value=['S_S (readily biodegradable substrate)',
           'S_O (oxygen)',
           'X_BH (active heterotrophic biomass)',
           'X_BA (active autotrophic biomass)'],
    rows=len(opts),
    description='State variables',
    disabled=False,
    style={'description_width': 'initial'},
    layout={'width':'50%'}
)
display(var_slt)

sim_btn = Button(
    description=' Simulate',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run simulation',
    icon='play' # (FontAwesome names without the `fa-` prefix)
)
display(sim_btn)

def run_interactive(btn):
    t = float(t_txt.value)
    t_step = float(t_step_txt.value)
    method = method_menu.value
    run(t, method, set_init=True, t_eval=np.arange(0, t+t_step, t_step))

    variables = var_slt.value
    plot(variables)
    
print('\n\n')
sim_btn.on_click(run_interactive)

Label(value='Simulation time [d]')

Text(value='1')

Label(value='Simulation time step [d]')

Text(value='0.05')

Dropdown(description='ODE solver:', index=3, options=('RK45', 'RK23', 'DOP853', 'BDF', 'LSODA'), value='BDF')

Label(value='Choose one or more (use shift and/or ctrl/cmd) state variables of interest:')

SelectMultiple(description='State variables', index=(1, 7, 4, 5), layout=Layout(width='50%'), options=('S_I (s…

Button(button_style='success', description=' Simulate', icon='play', style=ButtonStyle(), tooltip='Run simulat…